In [5]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar


from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial
import mthree
import qiskit.tools.jupyter

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

ibmqfactory.load_account:WARNING:2023-03-02 11:30:14,372: Credentials are already in use. The existing account in the session will be replaced.


Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [2]:
n_qubits = 7
n_meas_shots = 16000
n_build_shots = 16000

n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibm_oslo')

circuit = GHZ_prep(backend)

err_cmap = [[1, 3], [5, 6], [0, 5], [0, 4], [0, 2], [0, 3], [0, 6]]

## Qiskit Full Build

In [ ]:
n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

## Run with repetitions
If everything above is working, let's crank it out a few times

In [ ]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[]
}

for _ in range(n_reps):

    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
    tpf_err.build(verbose=True)
    
    tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
    tpf.build(verbose=True)
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        GHZ_state_dist(bare_result)
    )
    
    
    results['cmc'].append(
            GHZ_state_dist(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
            GHZ_state_dist(tpf_err.apply(bare_result))
    )
    results['aim'].append(GHZ_state_dist(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(GHZ_state_dist(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(GHZ_state_dist(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))



In [6]:
mit = mthree.M3Mitigation(backend)
mit.cals_from_system(list(range(n_qubits)), n_build_shots // 2)

ERROR:websocket:close status: 4002


In [9]:
dist_measure = GHZ_state_dist
n_reps = 3
for i in range(n_reps):
    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    bare_result = bare_result_job.result().get_counts()
    print(dist_measure(mit.apply_correction(bare_result, list(range(n_qubits))).nearest_probability_distribution()))

ERROR:websocket:close status: 4002


0.5122883866119126


ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f82e9137c40>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:close status: 4002


0.005033404185518453


ERROR:websocket:close status: 4002


0.011531869065367073


In [1]:
ghz_oslo_results = {'bare': [0.7698125, 0.7624375, 0.7776875000000001],
 'aim': [0.783625, 0.7827083333333333, 0.7809999999999999],
 'sim': [0.7711875, 0.7711250000000001, 0.77609375],
 'jigsaw': [0.7374128610389914, 0.8926451019481989, 0.5608884813432252],
 'cmc': [0.7536631803911169, 0.7517516308561187, 0.782489760135086],
 'cmc_err': [0.7441189816875593, 0.7314468107679665, 0.7635942542604933]}

In [ ]:
res = ghz_oslo_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')